https://mccormickml.com/2019/07/22/BERT-fine-tuning/

In [1]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
from util import input_maker
%matplotlib inline

from collections import Counter
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

from nltk.tokenize import sent_tokenize

Using TensorFlow backend.


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'GeForce GTX 1070'

In [3]:
NUM_CATEGORIES = 20

in_domain_train = r"D:\Data\cola\cola_public_1.1\cola_public\raw\in_domain_train.tsv"

In [4]:
data = pd.read_csv(r"D:\Data\wine-reviews\winemag-data-130k-v2.csv")
counter = Counter(data['variety'].tolist())
top_n_varieties = {i[0]: idx for idx, i in enumerate(counter.most_common(NUM_CATEGORIES))}
data = data[data['variety'].map(lambda x: x in top_n_varieties)]
description_list = data['description'].tolist() 
#description_list = ["[CLS] " + sentence + " [SEP]" for sentence in description_list]

{'Pinot Noir': 0,
 'Chardonnay': 1,
 'Cabernet Sauvignon': 2,
 'Red Blend': 3,
 'Bordeaux-style Red Blend': 4,
 'Riesling': 5,
 'Sauvignon Blanc': 6,
 'Syrah': 7,
 'Rosé': 8,
 'Merlot': 9,
 'Nebbiolo': 10,
 'Zinfandel': 11,
 'Sangiovese': 12,
 'Malbec': 13,
 'Portuguese Red': 14,
 'White Blend': 15,
 'Sparkling Blend': 16,
 'Tempranillo': 17,
 'Rhône-style Red Blend': 18,
 'Pinot Gris': 19}

In [5]:
#create a list of the index of the wine type
varietal_list = [top_n_varieties[i] for i in data['variety'].tolist()]
varietal_list = np.array(varietal_list)

In [6]:
description_list[0]

"[CLS] Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity. [SEP]"

In [44]:
sentences = []
labels = []

In [46]:
for index in range(len(description_list)):
    descriptions = sent_tokenize(description_list[index])
    for x in range(len(descriptions)):
        sentences.extend(descriptions[x])
    print(type(labels))
    labels = labels.append(15)

<class 'NoneType'>


AttributeError: 'NoneType' object has no attribute 'append'

In [40]:
labels = []
labels.extend([15])

In [41]:
varietal_list[0]

15

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenized_wine_texts = [tokenizer.tokenize(d) for d in description_list]

count = 0
for i in range(60): 
    d = tokenized_wine_texts[i]
    if d is None and count<10:
        print(d)
        count+=1

None in tokenized_wine_texts

False

In [8]:
sequence_lengths = [len(s) for s in tokenized_wine_texts]
print("the 98th percentile: ", np.percentile(sequence_lengths, 98))
print("max: ", np.max(sequence_lengths))

# Set the maximum sequence length. The longest sequence in our training set is 47, but we'll leave room on the end anyway. 
# In the original paper, the authors used a length of 512.
MAX_LEN = 64

# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_wine_texts]


#wine 
seqs = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_wine_texts]

for x in tokenized_wine_texts:
    if not tokenizer.convert_tokens_to_ids(x):
        print (x)

none_count = 0
#truncate so the max is 64
inpt_seqs = []
for seq in input_ids:
    new_seq = seq[:(MAX_LEN - 1)]
    new_seq.append(seq[-1])
    if not new_seq:
        #print("seq is none, ", seq)
        none_count +=1
    inpt_seqs.append(new_seq)

len(inpt_seqs)

# Pad our input tokens
input_ids = pad_sequences(inpt_seqs, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids.shape

# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

for i in range(5):
  print(np.sum(attention_masks[i]))

# Use train_test_split to split our data into train and validation sets for training

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, varietal_list, 
                                                            random_state=2018, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

# Convert all of our data into torch tensors, the required datatype for our model

train_inputs = torch.tensor(train_inputs).long()
print (train_inputs.type())
validation_inputs = torch.tensor(validation_inputs).long()
print (validation_inputs.type())
train_labels = torch.tensor(train_labels).long()
print (train_labels.type())
validation_labels = torch.tensor(validation_labels).long()
print (validation_labels.type())
train_masks = torch.tensor(train_masks).long()
print (train_masks.type())
validation_masks = torch.tensor(validation_masks).long()
print (validation_masks.type())

the 98th percentile:  98.0
max:  194
42.0
54.0
48.0
57.0
60.0
torch.LongTensor
torch.LongTensor
torch.LongTensor
torch.LongTensor
torch.LongTensor
torch.LongTensor


In [9]:
print (validation_inputs.shape)
print(validation_labels.shape)
print(validation_masks.shape)
print(train_inputs.shape)
print(train_labels.shape)
print(train_masks.shape)

torch.Size([9392, 64])
torch.Size([9392])
torch.Size([9392, 64])
torch.Size([84522, 64])
torch.Size([84522])
torch.Size([84522, 64])


In [10]:
#print(train_inputs)

In [11]:
# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 16

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [12]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", NUM_CATEGORIES)

In [13]:
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
   

In [14]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [15]:
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=2e-5,
                     warmup=.1)

t_total value of -1 results in schedule not being applied


In [16]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [20]:
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
  
  
  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()
  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    train_loss_set.append(loss.item())    
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    
    
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
    
  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Tracking variables 
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  # Evaluate data for one epoch
  for batch in validation_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))


Epoch:   0%|                                                                                    | 0/10 [00:00<?, ?it/s]

Train loss: 0.2301477236310438
Validation Accuracy: 0.7741695059625213



Epoch:  10%|███████▏                                                                | 1/10 [34:38<5:11:48, 2078.68s/it]

Train loss: 0.1712899690812315
Validation Accuracy: 0.778534923339012



Epoch:  20%|██████████████                                                        | 2/10 [1:09:14<4:37:02, 2077.78s/it]

...


Epoch:  80%|████████████████████████████████████████████████████████              | 8/10 [4:36:28<1:09:04, 2072.19s/it]

Train loss: 0.054332130183308
Validation Accuracy: 0.7728918228279387



Epoch:  90%|████████████████████████████████████████████████████████████████▊       | 9/10 [5:10:58<34:31, 2071.43s/it]

Train loss: 0.048629136541213105
Validation Accuracy: 0.7743824531516184



Epoch: 100%|███████████████████████████████████████████████████████████████████████| 10/10 [5:45:24<00:00, 2072.43s/it]


In [21]:
print(3)

3
